Notes for Jacopo:

In [21]:
kernel='j'
social='youtube'

if kernel!='Y':
  !git clone https://github.com/nudojacopo/thesis.git
  from google.colab import drive
  drive.mount('/content/drive')
import sys
module_path ='/content/thesis/src/PRO'
sys.path.append(module_path)
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from PRO_package import *
from scipy import stats
import random
from scipy.stats import chi2
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot

In [25]:
dataset='facebook_snews'
social_media_name = "facebook"
post_id = "post_id"
toxicity_score='toxicity_score'
comment_id='comment_id'
user='user_id'
created_at='created_at'
topic='topic'
social='social'

In [26]:
data = pd.read_csv('/content/drive/MyDrive/thesis/data/'+social_media_name+'/'+dataset+'.csv')

Filter mantaining just threads with more than 50 comments

In [23]:
comment_counts = data[post_id].value_counts()
threads_with_more_than_50_comments = comment_counts[comment_counts > 50].index
data = data[data[post_id].isin(threads_with_more_than_50_comments)]


## Stage 1 - Add variables ####


<br>
Essential to add some useful variables before to start working. This is the variables list:<br>
    <br>
* thread_lifetime <br>
* thread_birth<br>
* temporal_distance_birth<br>
* number_of_comments<br>
* number_of_users<br>
* number_of_comments_user_in_thread<br>
* sequence_number_comment_user_thread<br>
* inter arrival time user<br>
* inter arrival time user-thread<br>
* inter arrival time user-thread<br>


In [16]:
threads = data.groupby(post_id)  # Divide in threads

In [27]:
import pandas as pd

def add_percentile_column(group):
    group['percentile'] = (group['created_at'].rank(pct=True) * 100).round(2)
    return group

In [ ]:
progress_bar = tqdm(total=len(threads), desc="Add variables...")
for name, group in threads:
    group['time'] = pd.to_datetime(group['created_at'],format='ISO8601')
    group.sort_values(by='created_at', inplace=True)
    group['thread_birth'] = group['time'].min()
    group['temporal_distance_birth_h'] = round(pd.to_timedelta(group['time']-group['thread_birth']).dt.total_seconds() / 3600, 1)
    group['thread_lifetime_h'] = round((group['time'].max()-group['time'].min()).total_seconds() / 3600, 1)
    group['number_of_comments'] = len(group)
    group['number_of_users'] = len(group.user_id.unique())

    percentiles = add_percentile_column(group)['percentile']

    user_counts = group.groupby('user_id')['user_id'].count()
    group['number_of_comments_by_user_in_thread'] = group['user_id'].map(user_counts)
    group['user_appearances'] = group.groupby('user_id').cumcount() + 1

    data.loc[group.index, 'percentile'] = percentiles
    data.loc[group.index,'sequential_number_of_comment_by_user_in_thread'] = group['user_appearances']
    data.loc[group.index, 'number_of_comments_by_user_in_thread'] = group['number_of_comments_by_user_in_thread']
    data.loc[group.index, 'thread_birth'] = group['thread_birth']
    data.loc[group.index,'temporal_distance_birth_h'] = group['temporal_distance_birth_h']
    data.loc[group.index, 'thread_lifetime_h'] = group['thread_lifetime_h']
    data.loc[group.index, 'number_of_users'] = group['number_of_users']
    data.loc[group.index, 'number_of_comments'] = group['number_of_comments']
    progress_bar.update(1)

progress_bar.close()

Add variables...:  66%|██████▌   | 14056/21401 [04:23<01:57, 62.56it/s]

In [ ]:
data = data.sort_values(by=['user', 'created_at'])
data['created_at'] = pd.to_datetime(data['created_at'])
data['temporal_distance_from_previous_comment_h'] = data.groupby('user')['created_at'].diff().dt.total_seconds() / 3600

In [ ]:
data['root_submission']=data['root_submission'].astype('int')

In [ ]:
data['created_at'] = pd.to_datetime(data['created_at'])

In [ ]:
data = data.sort_values(by=['user', 'created_at'])
data['IAT_user'] = data.groupby('user')['created_at'].diff().dt.total_seconds()
data = data.sort_values(by=['user','root_submission', 'created_at'])
data['IAT_user_thread'] = data.groupby(['user','root_submission'])['created_at'].diff().dt.total_seconds()
data = data.sort_values(by=['root_submission', 'created_at'])
data['IAT_thread'] = data.groupby('root_submission')['created_at'].diff().dt.total_seconds()

In [ ]:
data = data.sort_values(by=['user','root_submission', 'created_at'])
data['IAT_user_thread'] = data.groupby(['user','root_submission'])['created_at'].diff().dt.total_seconds()

In [ ]:
data=data[[comment_id,user,post_id,  toxicity_score,
       created_at, 'percentile',
       'sequential_number_of_comment_by_user_in_thread',
       'number_of_comments_by_user_in_thread', 'thread_birth',
       'temporal_distance_birth_h', 'thread_lifetime_h', 'number_of_users',
       'unique_words_count', 'number_of_comments', 'language',
       'unique_word_user', 'temporal_distance_from_previous_comment_h',
       'IAT_user', 'IAT_user_thread', 'IAT_thread',topic,social]]

In [ ]:
data.to_csv(root+'src/PRO/output/'+social_media_name+'_processed.csv', index=False)